### Getting Setup

In [2]:
import torch
from torch import nn, optim
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import transforms
from torchinfo import summary
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, '../05/')
from going_modular import data_setup, engine
from going_modular.engine import train_step, test_step
import os
import zipfile
from pathlib import Path
import requests
from tqdm.auto import tqdm

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

#### Create a helper function to set seeds

In [4]:
def set_seeds(seed = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

### Get data

In [5]:
def download_data(source, destination, remove_source = True):
    data_path = Path('data/')
    img_path = data_path / destination
    if img_path.is_dir():
        print(f'[INFO] {img_path} directory exists, skipping download.')
    else:
        print(f'[INFO] Did not find {img_path} directory, creating one...')
        img_path.mkdir(parents = True, exist_ok = True)

        target_file = Path(source).name
        with open(data_path / target_file, 'wb') as f:
            request = requests.get(source)
            print(f'[INFO] Downloading {target_file} from {source}')
            f.write(request.content)

        # Unzip
        with zipfile.ZipFile(data_path / target_file, 'r') as zip_ref:
            print(f'[INFO] Unzipping {target_file} data...')
            zip_ref.extractall(img_path)
        
        if remove_source:
            os.remove(data_path / target_file)
    
    return img_path

img_path = download_data(
    source= 'https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip',
    destination='pizza_steak_sushi',
    remove_source=True
)

img_path

[INFO] data/pizza_steak_sushi directory exists, skipping download.


PosixPath('data/pizza_steak_sushi')

### Create Datasets and DataLoaders

#### Create DataLoaders using manually created transforms

In [6]:
train_dir = img_path / 'train'
test_dir = img_path / 'test'

normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

manual_transforms = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    normalize
])

print(f'Manually created transforms: {manual_transforms}')

train_loader, test_loader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    transform = manual_transforms,
    batch_size = 32
)

train_loader, test_loader, class_names

Manually created transforms: Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


(<torch.utils.data.dataloader.DataLoader at 0x164f4fb50>,
 ['pizza', 'steak', 'sushi'])

#### Create DataLoaders using automatically created transforms

In [7]:
train_dir = img_path / 'train'
test_dir = img_path / 'test'

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

automatic_transforms = weights.transforms()
print(f'Automatically created transforms: {automatic_transforms}')

train_loader, test_loader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    transform = automatic_transforms,
    batch_size = 32
)

train_loader, test_loader, class_names

Automatically created transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x1658cd5d0>,
 ['pizza', 'steak', 'sushi'])

### Getting a pretrained model, freezing base layers and changing classifier head

In [8]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [9]:
# Freeze all base layers
for param in model.features.parameters():
    param.requires_grad = False

set_seeds()

model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=len(class_names), bias=True).to(device)
)

In [10]:
summary(
    model = model,
    input_size= (32, 3, 224, 224),
    verbose=0,
    col_names= ['input_size', 'output_size', 'num_params', 'trainable'],
    col_width=20,
    row_settings=['var_names']
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

### Train model and track results

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.001)

In [12]:
writer = SummaryWriter()

In [13]:
def train(model, train_loader, test_loader, optimizer, loss_fn, epochs, device):
    results = {
        'train_loss': [],
        'train_acc': [],
        'test_loss': [],
        'test_acc': []
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model, train_loader, loss_fn, optimizer, device
        )
        test_loss, test_acc = test_step(
            model, test_loader, loss_fn, device
        )

        print(f'Epoch: {epoch + 1}\ntrain_loss: {train_loss}\ntrain_acc: {train_acc}\ntest_loss: {test_loss}\ntest_acc: {test_acc}')

        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)

        writer.add_scalars(main_tag='Loss', tag_scalar_dict={
            'train_loss': train_loss,
            'test_loss': test_loss
        }, global_step=epoch)

        writer.add_scalars(main_tag='Accuracy', tag_scalar_dict={
            'train_acc': train_acc,
            'test_acc': test_acc
        }, global_step=epoch)

        writer.add_graph(model=model, input_to_model=torch.randn(32, 3, 224, 224).to(device))

    writer.close()

    return results

In [14]:
set_seeds()

results = train(
    model, train_loader, test_loader, optimizer, loss_fn, 5, device
)

  0%|          | 0/5 [00:00<?, ?it/s]

/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/hom

Epoch: 1
train_loss: 1.0882934033870697
train_acc: 0.41796875
test_loss: 0.8914491534233093
test_acc: 0.6818181818181818


 20%|██        | 1/5 [01:34<06:18, 94.53s/it]/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  

Epoch: 2
train_loss: 0.8936691954731941
train_acc: 0.6640625
test_loss: 0.8082305391629537
test_acc: 0.774621212121212


...

 60%|██████    | 3/5 [04:41<03:07, 93.79s/it]/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  

Epoch: 4
train_loss: 0.7797179818153381
train_acc: 0.69921875
test_loss: 0.6849217216173807
test_acc: 0.8039772727272728


 80%|████████  | 4/5 [06:14<01:33, 93.60s/it]/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  

Epoch: 5
train_loss: 0.6321720890700817
train_acc: 0.76953125
test_loss: 0.6428378423055013
test_acc: 0.8361742424242425


100%|██████████| 5/5 [07:49<00:00, 93.83s/it]


In [15]:
results

{'train_loss': [1.0882934033870697,
  0.8936691954731941,
  0.744956448674202,
  0.7797179818153381,
  0.6321720890700817],
 'train_acc': [0.41796875, 0.6640625, 0.84375, 0.69921875, 0.76953125],
 'test_loss': [0.8914491534233093,
  0.8082305391629537,
  0.7433454990386963,
  0.6849217216173807,
  0.6428378423055013],
 'test_acc': [0.6818181818181818,
  0.774621212121212,
  0.7537878787878788,
  0.8039772727272728,
  0.8361742424242425]}

### View model results in TensorBoard

In [18]:
%tensorboard --logdir=runs/ --host localhost --port 8080